In [1]:
#include "xeus/xcomm.hpp"
#include "xproperty/xobserved.hpp"

#include <string>
#include <iostream>

   
inline void xwidget_comm_opened(const xeus::xcomm& comm, const xeus::xmessage& msg)
{
    std::cout << "Hello World" << std::endl;
}

In [2]:
auto& interpreter = xeus::get_interpreter();
interpreter.comm_manager().register_comm_target("jupyter.widget", xwidget_comm_opened);

auto& manager = interpreter.comm_manager();
auto target = manager.target("jupyter.widget");

In [3]:
namespace xeus
{

    template <class D>
    class xobject : public xp::xobserved<D>
    {
    public:

        xobject() : m_comm(get_interpreter().comm_manager().target(target_name), xguid())
        {
            set_defaults();
        }
        
        inline void set_defaults()
        {
            _model_module.raw_value() = "jupyter-js-widgets";
            _model_module_version.raw_value() = "~2.1.4";
            _model_name.raw_value() = "WidgetModel";
            _view_module.raw_value() = "jupyter-js-widgets";
            _view_module_version.raw_value() = "~2.1.4";
            _view_name.raw_value() = "WidgetView";
        }

        inline xjson get_state() const
        {
            xjson state;
            state["_model_module"] = _model_module.raw_value();
            state["_model_module_version"] = _model_module_version.raw_value();
            state["_model_name"] = _model_name.raw_value();
            state["_view_module"] = _view_module.raw_value();
            state["_view_module_version"] = _view_module_version.raw_value();
            state["_view_name"] = _view_name.raw_value();
            return state;
        }

        inline xguid id() const noexcept
        {
            return m_comm.id();
        }

        // TODO: make these property optional (allow none)
        XPROPERTY(std::string, D, _model_module);
        XPROPERTY(std::string, D, _model_module_version);
        XPROPERTY(std::string, D, _model_name);
        XPROPERTY(std::string, D, _view_module);
        XPROPERTY(std::string, D, _view_module_version);
        XPROPERTY(std::string, D, _view_name);

        inline void display() const
        {
            xeus::xjson display_data = R"(
            {
                "application/vnd.jupyter.widget-view+json": {
                    "version_major": "2",
                    "version_minor": "0"
                }
            }
            )"_json;

            display_data["application/vnd.jupyter.widget-view+json"]["model_id"] = 
                xeus::guid_to_hex(this->derived_cast().id());
            
            get_interpreter().display_data(
                std::move(display_data),
                xeus::xjson::object(),
                xeus::xjson::object()
            );
            
            xeus::xjson comm_display = R"(
            {
                "data": {
                    "method": "display"
                }
            }
            )"_json;

            comm_display["comm_id"] =  xeus::guid_to_hex(this->derived_cast().id());

            get_interpreter().comm_manager().target(target_name)->publish_message("comm_msg", xeus::xjson::object(), std::move(comm_display));  
        }

    protected:

        xcomm m_comm;

        static const char* target_name;
    };

    // In the cpp (multiple inheritance)
    template <class D>
    const char* xeus::xobject<D>::target_name = "jupyter.widget";
    
    template <class D>
    void to_json(xjson& j, const xobject<D>& o)
    {
        j = "IPY_MODEL_" + guid_to_hex(o.id());
    }
}

In [4]:
namespace xeus
{
    template <class T>
    class slider : public xobject<slider<T>>
    {
    public:

        using base_type = xobject<slider<T>>;

        slider() : base_type()
        {
            set_defaults();
            base_type::m_comm.open(xjson::object(), get_state());
        }
    
        XPROPERTY(double, slider, value);

        inline void set_defaults()
        {
            base_type::_model_name.raw_value() = "FloatSliderModel";
            base_type::_view_name.raw_value() = "FloatSliderView";
        }

        inline xjson get_state() const
        {
            xjson state = base_type::get_state();
            state["value"] = value.raw_value();
            return state;
        }
    };
}

In [5]:
xeus::slider<double> slider;
slider.display()